In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
import src.config as config

In [3]:
import src.config as config
from src.inference import (
    get_feature_store
)

from datetime import datetime, timezone, timedelta

def get_current_utc_hour_ceiled():
    """Get current UTC time ceiled to the next hour"""
    current = datetime.now(timezone.utc)
    if current.minute > 0 or current.second > 0:
        # Ceil to next hour
        current = current + timedelta(hours=1)

    # Reset minutes, seconds, and microseconds
    current = current.replace(minute=0, second=0, microsecond=0)
    return current

# Usage
current_hour = get_current_utc_hour_ceiled()
print(f"Current UTC hour (ceiled): {current_hour}")

Current UTC hour (ceiled): 2025-05-11 16:00:00+00:00


In [4]:
def fetch_predictions():
    current_date = pd.Timestamp.now(tz='Etc/UTC').ceil('h')

    fs = get_feature_store()
    fg = fs.get_feature_group(
        name=config.FEATURE_GROUP_MODEL_PREDICTION,
        version=1
    )

    query = fg.select_all()
    query = query.filter(fg.pickup_hour == current_hour)

    return query.read()

In [5]:
fs = get_feature_store()
fg = fs.get_feature_group(
    name=config.FEATURE_GROUP_MODEL_PREDICTION,
    version=1
)

query = fg.select_all()

2025-05-11 11:04:53,988 INFO: Initializing external client
2025-05-11 11:04:53,988 INFO: Base URL: https://c.app.hopsworks.ai:443




To ensure compatibility please install the latest bug fix release matching the minor version of your backend (4.2) by running 'pip install hopsworks==4.2.*'


2025-05-11 11:04:55,965 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1214697


In [6]:
df= query.read()

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.51s) 


In [7]:
df

,pickup_location_id,predicted_demand,pickup_hour
0,5779,100.0,2025-05-11 12:00:00+00:00
1,5905,73.0,2025-05-11 12:00:00+00:00
2,6072,39.0,2025-05-11 12:00:00+00:00
3,5626,162.0,2025-05-11 12:00:00+00:00
4,5788,104.0,2025-05-11 12:00:00+00:00
5,5788,103.0,2025-05-11 13:00:00+00:00
6,5905,101.0,2025-05-11 13:00:00+00:00
7,5779,163.0,2025-05-11 13:00:00+00:00
8,6072,58.0,2025-05-11 13:00:00+00:00
9,5626,156.0,2025-05-11 13:00:00+00:00


In [ ]:
from datetime import datetime, timezone, timedelta

def fetch_next_hour_predictions():
    # Get current UTC time and round up to next hour
    now = datetime.now(timezone.utc)
    next_hour = (now + timedelta(hours=1)).replace(minute=0, second=0, microsecond=0)
    next_hour = pd.Timestamp(next_hour).ceil("6H")

    # Extract components from the rounded-up time
    year = next_hour.year
    month = next_hour.month
    day = next_hour.day
    hour = next_hour.hour

    # Create date strings in YYYY-MM-DD format
    current_date = f"{year}-{month:02d}-{day:02d}"
    next_date = next_hour + timedelta(days=1)
    next_date_str = next_date.strftime("%Y-%m-%d")

    fs = get_feature_store()
    fg = fs.get_feature_group(
        name=config.FEATURE_GROUP_MODEL_PREDICTION,
        version=1
    )

    # # First get the full day's data from Hopsworks
    # df = fg.filter(
    #     (fg.pickup_hour >= current_date) &
    #     (fg.pickup_hour < next_date_str)
    # ).read()

    # Then filter for next hour in the DataFrame
    hour_str = f"{current_date} {hour:02d}:00"
    df_hour = df[df['pickup_hour'] == hour_str]

    print(f"Current UTC time: {now}")
    print(f"Next hour: {next_hour}")
    print(f"Querying for date range: {current_date} to {next_date_str}")
    print(f"Filtering for hour: {hour_str}")
    print(f"Found {len(df_hour)} records")
    return df_hour

# Example usage:
predictions = fetch_next_hour_predictions()

AttributeError: 'datetime.datetime' object has no attribute 'dt'

In [107]:
now = datetime.now(timezone.utc)

In [108]:
predictions

,pickup_location_id,predicted_demand,pickup_hour
2763,188,1.0,2025-03-05 11:00:00+00:00
2764,259,0.0,2025-03-05 11:00:00+00:00
2765,189,0.0,2025-03-05 11:00:00+00:00
2766,229,117.0,2025-03-05 11:00:00+00:00
2767,21,0.0,2025-03-05 11:00:00+00:00
...,...,...,...
3010,80,2.0,2025-03-05 11:00:00+00:00
3011,124,0.0,2025-03-05 11:00:00+00:00
3012,119,0.0,2025-03-05 11:00:00+00:00
3013,23,0.0,2025-03-05 11:00:00+00:00


In [109]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3015 entries, 0 to 3014
Data columns (total 3 columns):
 #   Column              Non-Null Count  Dtype                  
---  ------              --------------  -----                  
 0   pickup_location_id  3015 non-null   int32                  
 1   predicted_demand    3015 non-null   float64                
 2   pickup_hour         3015 non-null   datetime64[us, Etc/UTC]
dtypes: datetime64[us, Etc/UTC](1), float64(1), int32(1)
memory usage: 59.0 KB


In [110]:
import pandas as pd
current_date = pd.Timestamp.now(tz='Etc/UTC').ceil('h')

In [111]:
dt = current_date.ceil('h')


In [112]:
fs = get_feature_store()  
fg = fs.get_feature_group(  
    name=config.FEATURE_GROUP_MODEL_PREDICTION,  
    version=1  
)  
query = fg.select_all() 
# query = query.filter(fg.pickup_hour >= "2025-02-13 04:00")
query = query.filter(fg.pickup_hour > "2025-02-13 05:00:00")  
results = query.read()

2025-03-05 05:08:13,346 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-05 05:08:13,360 INFO: Initializing external client
2025-03-05 05:08:13,360 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-05 05:08:14,014 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1214697
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (4.30s) 


In [113]:
fg.filter((fg.pickup_hour >= "2025-02-13") & (fg.pickup_hour < "2025-02-14")).read()

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.46s) 


,pickup_location_id,predicted_demand,pickup_hour


In [114]:
results

,pickup_location_id,predicted_demand,pickup_hour
0,127,0.0,2025-03-04 10:00:00+00:00
1,146,2.0,2025-03-04 10:00:00+00:00
2,106,0.0,2025-03-04 10:00:00+00:00
3,260,2.0,2025-03-04 10:00:00+00:00
4,174,0.0,2025-03-04 10:00:00+00:00
...,...,...,...
3010,80,2.0,2025-03-05 11:00:00+00:00
3011,124,0.0,2025-03-05 11:00:00+00:00
3012,119,0.0,2025-03-05 11:00:00+00:00
3013,23,0.0,2025-03-05 11:00:00+00:00


In [115]:
def fetch_next_hour_predictions():
    # Get current UTC time and round up to next hour
    now = datetime.now(timezone.utc)
    next_hour = (now + timedelta(hours=1)).replace(minute=0, second=0, microsecond=0)

    fs = get_feature_store()
    fg = fs.get_feature_group(name=config.FEATURE_GROUP_MODEL_PREDICTION, version=1)

    # Then filter for next hour in the DataFrame
    df_hour = df[df["pickup_hour"] == next_hour]

    print(f"Current UTC time: {now}")
    print(f"Next hour: {next_hour}")
    print(f"Found {len(df_hour)} records")
    return df_hour

In [116]:
predictions = fetch_next_hour_predictions()

2025-03-05 05:08:25,292 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-05 05:08:25,302 INFO: Initializing external client
2025-03-05 05:08:25,303 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-05 05:08:25,968 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1214697
Current UTC time: 2025-03-05 10:08:25.292157+00:00
Next hour: 2025-03-05 11:00:00+00:00
Found 252 records


In [117]:
fs = get_feature_store()
fg = fs.get_feature_group(name=config.FEATURE_GROUP_MODEL_PREDICTION, version=1)

2025-03-05 05:08:26,812 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-05 05:08:26,816 INFO: Initializing external client
2025-03-05 05:08:26,816 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-05 05:08:27,396 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1214697


In [118]:
df = fg.read()

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.74s) 


In [119]:
df["pickup_hour"].max()

Timestamp('2025-03-05 11:00:00+0000', tz='Etc/UTC')

In [120]:
now = datetime.now(timezone.utc)
next_hour = (now + timedelta(hours=1)).replace(minute=0, second=0, microsecond=0)
print(next_hour)

2025-03-05 11:00:00+00:00


In [121]:
df[df["pickup_hour"] == next_hour]

,pickup_location_id,predicted_demand,pickup_hour
2763,188,1.0,2025-03-05 11:00:00+00:00
2764,259,0.0,2025-03-05 11:00:00+00:00
2765,189,0.0,2025-03-05 11:00:00+00:00
2766,229,117.0,2025-03-05 11:00:00+00:00
2767,21,0.0,2025-03-05 11:00:00+00:00
...,...,...,...
3010,80,2.0,2025-03-05 11:00:00+00:00
3011,124,0.0,2025-03-05 11:00:00+00:00
3012,119,0.0,2025-03-05 11:00:00+00:00
3013,23,0.0,2025-03-05 11:00:00+00:00
